# Ground State Preparation of SU(2) Lattice Gauge Theory in Quantum Computer

[Kogut-Susskind Hamiltonian](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.11.395) for SU(2) Gauge Theory is:
$$
H = \frac{g^2}{2}\sum_{\rm links} (E_i^a)^2 - \frac{2}{a^2g^2} \sum_{\rm plaquettes} Z({\bf n}) \,,
$$
where, a in the denominator is the lattice spacing and in the superscript denotes SU(2) gauge group indices that are implicitly summer over, g is the gauge coupling constant with mass dimension $[g] = 0.5$ in 2+1 dimensions, $i = x$ or $y$ denotes spatial directions (implicitly summed), $\bf{n} = (n_x, n_y)$ is a a lattice point and $Z(\bf{n})$ is the plaquette operator, $E_i^a$ is the electric field along $i$th spatial direction associated with gauge group index $a$. 

**Mapping onto Spin Chain:**
The Hamiltonian of the SU(2) gauge theory on a plaquette chain with a basis truncated at $j=1/2$ can be mapped onto a quantum spin chain, shown in 
$$ H = \frac{3}{2}g^2\sum_{i=0}^{N-1}\frac{\sigma_i^z+1}{2} - \frac{3}{4}g^2\sum_{i=0}^{N-1}\frac{\sigma_i^z+1}{2}\frac{\sigma_{i+1}^z+1}{2} - \frac{2}{a^2g^2} \sum_{i=0}^{N-1} \big(-0.5\big)^{\frac{\sigma_{i-1}^z+\sigma_{i+1}^z+ 2}{2}}
\sigma_i^x \,.$$
Up to an irrelevant constant, this Hamiltonian can be rewritten as (see Ref. [1](https://arxiv.org/abs/2103.05179) and [2](https://arxiv.org/abs/2205.09247)} for a similar expression)
$$ H_{tot} = J \sum_{i=0}^{N-1}\sigma_i^z\sigma_{i+1}^z + h_z \sum_{i=0}^{N-1}\sigma_i^z + h_x \sum_{i=0}^{N-1} \frac{1-3\sigma_{i-1}^z}{4} \frac{1-3\sigma_{i+1}^z}{4} \sigma_i^x \, ,$$

That is,
$$ H_{tot} = J \sum_{i=0}^{N-1}\sigma_i^z\sigma_{i+1}^z + h_z \sum_{i=0}^{N-1}\sigma_i^z + \frac{h_x}{16} \sum_{i=0}^{N-1} ( \sigma_i^x - 3\sigma_{i-1}^z \sigma_i^x - 3\sigma_i^x\sigma_{i+1}^z + 9 \sigma_{i-1}^z \sigma_i^x \sigma_{i+1}^z )$$


where $J = -3 g^2/16$, $h_z=3 g^2/8$ and $h_x = -2/(ag)^2$. Under the **periodic boundary condition**, $\sigma_N^i=\sigma_0^i$. The Hamiltonian is rescaled to be unitless and so are the parameters $J$, $h_z$ and $h_x$.

## ADAPT-VQE Algorithm for the Interacting Ground State Preparation

Here, we plan to use ADAPT-VQE algorithm for the ground state preparation of the SU(2) gauge theory in the quantum computer. Find reference here, [ADAPT (Adaptive Derivative-Assembled Problem-Tailored)](https://www.nature.com/articles/s41467-019-10988-2).

The total hamiltonian in terms of Pauli $X, Z$ gates is:

$$ H_{tot} = \sum_{i=0}^{N-1}  [J Z_i Z_{i+1} + h_z Z_i + \frac{h_x}{16} ( X_i - 3 Z_{i-1} X_i - 3 X_i Z_{i+1} + 9 Z_{i-1} X_i Z_{i+1})]$$

where, $J = -3 g^2/16$, $h_z=3 g^2/8$ and $h_x = -2/(ag)^2$. Under the **periodic boundary condition**, $\sigma_N^i=\sigma_0^i$. The Hamiltonian is rescaled to be unitless and so are the parameters $J$, $h_z$ and $h_x$.


**ADAPT-VQE Algorithm:**

**Step 1:** Construct a pool of operators $\{\hat{A}^{(1)}, \hat{A}^{(2)}, \hat{A}^{(3)}, \hat{A}^{(4)}, \cdots\}$ constrained by the symmetry of the system.

**Step 2:** On the quantum device, initialize the quantum circuit the the current ansatz $\ket{\psi_{ansatz}}$ with desired quantum numbers and the symmetries of the target. The ansatz is dynamically created: $\cdots e^{\theta_3A_3}e^{\theta_2A_2}e^{\theta_1A_1} \ket{\psi_{ref}}$.

**Step 3:** Each time add one operator that gives the largest gradient magnitude.

That is, measure the energy gradient $\frac{\partial E}{\partial \theta_i} |_{\theta_i=0}$ with respect to the variational parameter $\theta_i$ of the candidate pool operator $\hat{A}$. Repeat this step for every pool operator. Here, we measure the expectation value of the commutator of hamiltonian with each operator in the pool $\braket{\psi^{(k)}_{ansatz}|[\hat{H},\hat{A_i}]|\psi^{(k)}_{ansatz}} = \frac{\partial E}{\partial \theta_i} |_{\theta_i=0} = [\frac{\partial }{\partial \theta_i} \braket{\psi^{(k)}_{ansatz}|e^{-\theta_i A_i}H e^{\theta_i A_i}|\psi^{(k)}_{ansatz}}]|_{\theta_i=0}$ which gives the estimate of decrease in the energy by transforming the ansatz wavefunction from $\ket{\psi_{ansatz}} \rightarrow e^{i \theta_i \hat{A}}\ket{\psi_{ansatz}}$.

That is to say, add the operator $\hat{A}_n$ with the largest gradient norm to the ansatz with its variational parameter set to zero.

**Step 4:** Perform ordinary VQE to update all ansatz parameters. 

**Step 6:** Repeat steps 1 to 4 until convergence. That is if the measured expectation value is less than the tolerance/threshold ($\epsilon$), then algorithm terminates as the ADAPT-VQE has converged.


After adding the operators, after k-th iteration, the ansatz has the form:

$\ket{\psi_k} = e^{\theta_k A_k}.....e^{\theta_2 A_2} e^{\theta_1 A_1} \ket{\psi_0} $,

and the energy gradiant with respect to the variational parameter of the candidate operator $\hat{A}_i$, $\theta_i$ in the $(k+1)-th$ iteration, using the antihermiticity of the pool operators becomes


In our case, we can construct the operators out of our hamiltonian. 

Our aim is to find an efficient partitioning of the commutators of many pool operators and many Hamiltonian terms, assuming that we have a decomposition of both as sums of Pauli words. As usual, by Pauli words (also known as Pauli strings) we mean tensor products of Pauli matrices (including the identity), i.e. $\hat{A} = a_1 \bigotimes a_2 \bigotimes \cdot $ where $a_i \in \{I, X, Z\}$, and to avoid notational clutter we omit identities and the tensor product symbol and add qubit indices everywhere else, e.g. $X_2 Z_3$ acting on the Hilbert space of 4 qubits should be read as $ I \bigotimes X \bigotimes Z \bigotimes I$. Now consider the commutator of two commutators, $[[P, S_i ], [P, S_j ]]$ for Pauli words $P , S_i$ , and $S_j$. If either of the inner commutators vanish, the outer one vanishes trivially. Considering only non-vanishing inner commutators and using the fact that Pauli words that do not commute anticommute, we have $[[P, S_i ], [P, S_j ]] = 4[P S_i , P S_j ] = 4(−S_i P P S_j + S_j P P S_i )= 4[S_j , S_i ]$.

This implies that $[[P, S_i ], [P, S_j]]$ vanishes if $S_i$ and $S_j$ commute, and by extension, the commutators $[P, S_k]$ of all elements $S_k$ of mutually commuting set $S$ with any Pauli word $P$ commute. Since all commutators revolve around $P$, from this point onward we call $P$ the pivot of the set. Since we are interested in measuring the commutators of the Hamiltonian and each pool operator, we have the freedom to choose the pivots to be operators from our pool, or the individual terms in the Hamiltonian.

In [1]:
# ## INSTALLATIONS REQUIRED
# !pip install qiskit[visualization]==1.1.0
# !pip install qiskit_aer
# !pip install qiskit qiskit-aer
# !pip install scipy
# !pip install numpy
# !pip install matplotlib
# !pip install qiskit-ibm-runtime
# !pip install -U sympy
# !pip install distinctipy
# !pip install pylatexenc
# !pip install prototype-zne
# !pip install physics-tenpy

In [2]:
import qiskit
qiskit.__version__

'1.1.1'

In [3]:
#import necessary libraries
import numpy as np
import scipy.linalg as LA
from scipy.linalg import eig, eigh
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from qiskit import *
import matplotlib.pyplot as plt
import distinctipy
import matplotlib.ticker as ticker
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.quantum_info import Statevector
from qiskit.visualization import state_visualization
from qiskit.circuit import QuantumCircuit, Parameter
import qiskit.quantum_info as qi
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2, Batch
from qiskit_ibm_runtime.options import EstimatorOptions, DynamicalDecouplingOptions
from qiskit.transpiler import CouplingMap
from qiskit.primitives import StatevectorEstimator, Estimator
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.primitives import BackendEstimator
from qiskit.providers.fake_provider import GenericBackendV2

## DMRG
# import tenpy
# from tenpy.networks.mps import MPS
# from tenpy.models.lattice import Chain
# from tenpy.models.spins import SpinModel
# from tenpy.algorithms.dmrg import dmrg
# from tenpy.tools.params import get_parameter

In [4]:
### HAMILTONIAN WITH PERIODIC BOUNDARY CONDITIONS

## The electric part of the Hamiltonian
def obs_z_terms(N, J, hz):
    """
    Parameters: 
    N (int): Number of spins
    J (float): constant
    hz (float): another constant

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []
    
    # Constructing the linear terms hz * sum(sigma_i^z)
    for i in range(N):
        z_term = ['I'] * N
        z_term[i] = 'Z'
        pauli_list.append(Pauli(''.join(z_term))) 
        coeffs.append(hz)

    return SparsePauliOp(pauli_list, coeffs)   

def obs_zz_terms(N, J, hz):
    """
    Parameters: 
    N (int): Number of spins
    J (float): constant
    hz (float): another constant

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    # Constructing the interaction terms J * sum(sigma_i^z * sigma_{i+1}^z)
    for i in range(N):
        z_term = ['I'] * N
        z_term[i] = 'Z'
        z_term[(i + 1) % N] = 'Z'
        pauli_list.append(Pauli(''.join(z_term))) 
        coeffs.append(J)
    return SparsePauliOp(pauli_list, coeffs)

 
## The Magnetic parts of the hamiltonian
def obs_x_terms(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16

    for i in range(N):
        # Term: sigma_i^x
        x_term = ['I'] * N
        x_term[i] = 'X'
        pauli_list.append(Pauli(''.join(x_term)))
        coeffs.append(factor)
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_zx_terms(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    
    for i in range(N):
        # Term: -3 * sigma_{i-1}^z * sigma_i^x
        zx_term = ['I'] * N
        zx_term[(i-1) % N] = 'Z'
        zx_term[i] = 'X'
        pauli_list.append(Pauli(''.join(zx_term)))
        coeffs.append(-3 * factor)
        
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_xz_terms(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    for i in range(N):    
        # Term: -3 * sigma_i^x * sigma_{i+1}^z
        xz_term = ['I'] * N
        xz_term[i] = 'X'
        xz_term[(i+1) % N] = 'Z' #periodic boundary condition (imposed by % sign)
        pauli_list.append(Pauli(''.join(xz_term)))
        coeffs.append(-3 * factor)
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_zxz_terms(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    for i in range(N):
        # Term: 9 * sigma_{i-1}^z * sigma_i^x * sigma_{i+1}^z
        zxz_term = ['I'] * N
        zxz_term[(i-1) % N] = 'Z'
        zxz_term[i] = 'X'
        zxz_term[(i+1) % N] = 'Z'
        pauli_list.append(Pauli(''.join(zxz_term)))
        coeffs.append(9 * factor)

    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

In [15]:
### HAMILTONIANS WITH OPEN BOUNDARY CONDITIONS

# Create the electric part of the Hamiltonian with open boundary conditions
def obs_z_terms_open(N, J, hz):
    """
    Parameters: 
    N (int): Number of spins
    J (float): constant
    hz (float): another constant

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []
    
    # Constructing the linear terms hz * sum(sigma_i^z)
    for i in range(N):
        z_term = ['I'] * N
        z_term[i] = 'Z'
        pauli_list.append(Pauli(''.join(z_term))) 
        coeffs.append(hz)

    return SparsePauliOp(pauli_list, coeffs)   

def obs_zz_terms_open(N, J, hz):
    """
    Parameters: 
    N (int): Number of spins
    J (float): constant
    hz (float): another constant

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    # Constructing the interaction terms J * sum(sigma_i^z * sigma_{i+1}^z)
    for i in range(N - 1):  # Change N to N-1 to avoid wrapping around
        z_term = ['I'] * N
        z_term[i] = 'Z'
        z_term[i + 1] = 'Z'
        pauli_list.append(Pauli(''.join(z_term))) 
        coeffs.append(J)
    return SparsePauliOp(pauli_list, coeffs)

# Create the magnetic part of the Hamiltonian with open boundary conditions
def obs_x_terms_open(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16

    for i in range(N):
        # Term: sigma_i^x
        x_term = ['I'] * N
        x_term[i] = 'X'
        pauli_list.append(Pauli(''.join(x_term)))
        coeffs.append(factor)
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_zx_terms_open(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    
    for i in range(1, N):  # Start from 1 to avoid the negative index
        # Term: -3 * sigma_{i-1}^z * sigma_i^x
        zx_term = ['I'] * N
        zx_term[i - 1] = 'Z'
        zx_term[i] = 'X'
        pauli_list.append(Pauli(''.join(zx_term)))
        coeffs.append(-3 * factor)
        
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_xz_terms_open(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    for i in range(N - 1):  # Change N to N-1 to avoid wrapping around    
        # Term: -3 * sigma_i^x * sigma_{i+1}^z
        xz_term = ['I'] * N
        xz_term[i] = 'X'
        xz_term[i + 1] = 'Z'
        pauli_list.append(Pauli(''.join(xz_term)))
        coeffs.append(-3 * factor)
    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

def obs_zxz_terms_open(N, hx):
    """
    Parameters:
    N (int): Number of spins
    hx (float): Constant parameter hx

    Returns:
    SparsePauliOp: SparsePauliOp object 
    """
    pauli_list = []
    coeffs = []

    factor = hx / 16
    for i in range(1, N - 1):  # Change range to avoid boundary issues
        # Term: 9 * sigma_{i-1}^z * sigma_i^x * sigma_{i+1}^z
        zxz_term = ['I'] * N
        zxz_term[i - 1] = 'Z'
        zxz_term[i] = 'X'
        zxz_term[i + 1] = 'Z'
        pauli_list.append(Pauli(''.join(zxz_term)))
        coeffs.append(9 * factor)

    # Create SparsePauliOp 
    return SparsePauliOp(pauli_list, coeffs)

In [25]:
# CONSTRUCT TOTAL HAMILTONIAN
def construct_total_hamiltonian(N, J, hz, hx):
    H_z = obs_z_terms(N, J, hz)
    H_zz = obs_zz_terms(N, J, hz)
    H_x = obs_x_terms(N, hx)
    H_zx = obs_zx_terms(N, hx)
    H_xz = obs_xz_terms(N, hx)
    H_zxz = obs_zxz_terms(N, hx)
    H_tot = H_z + H_zz + H_x + H_zx + H_xz + H_zxz
    return H_tot

In [26]:
## initial reference state 
def initial_reference_state(N):
    qc = QuantumCircuit(N)
    return qc

In [27]:
### TESTING THE ABOVE FUNCTIONS ####
# Parameters
N = 5 #spins

a = 1.0 #lattice
g = 0.8 #coupling

J = -3 * g**2 / 16
hz = 3 * g**2 / 8
hx = -2 / (a * g)**2  

# Construct the terms of the Hamiltonian
z_terms = obs_z_terms(N, J, hz)
zz_terms = obs_zz_terms(N, J, hz)
x_terms = obs_x_terms(N, hx)
zx_terms = obs_zx_terms(N, hx)
xz_terms = obs_xz_terms(N, hx)
zxz_terms = obs_zxz_terms(N, hx)

# Function to check commutation
def check_commutation(mat1, mat2):
    commutator = mat1 @ mat2 - mat2 @ mat1
    commutator_matrix = commutator.to_matrix()
    return np.allclose(commutator_matrix, np.zeros_like(commutator_matrix))
# Find commutation relation
def commutator(H_tot, obs):
    """Compute [H_tot, obs]"""
    return H_tot @ obs - obs @ H_tot
    
# List of all terms
terms = [z_terms, zz_terms, x_terms, zx_terms, xz_terms, zxz_terms]
term_names = ['Z terms', 'ZZ terms', 'X terms', 'ZX terms', 'XZ terms', 'ZXZ terms']

# Check commutativity between all pairs
for i in range(len(terms)):
    for j in range(i, len(terms)):
        term1 = terms[i]
        term2 = terms[j]
        commutes = check_commutation(term1, term2)
        print(f"{term_names[i]} and {term_names[j]} commute: {commutes}")

Z terms and Z terms commute: True
Z terms and ZZ terms commute: True
Z terms and X terms commute: False
Z terms and ZX terms commute: False
Z terms and XZ terms commute: False
Z terms and ZXZ terms commute: False
ZZ terms and ZZ terms commute: True
ZZ terms and X terms commute: False
ZZ terms and ZX terms commute: False
ZZ terms and XZ terms commute: False
ZZ terms and ZXZ terms commute: False
X terms and X terms commute: True
X terms and ZX terms commute: False
X terms and XZ terms commute: False
X terms and ZXZ terms commute: False
ZX terms and ZX terms commute: True
ZX terms and XZ terms commute: True
ZX terms and ZXZ terms commute: False
XZ terms and XZ terms commute: True
XZ terms and ZXZ terms commute: False
ZXZ terms and ZXZ terms commute: True


In [32]:
# Combine commuting terms into pools
pool1 = z_terms + zz_terms
pool2 = x_terms
pool3 = zx_terms + xz_terms
pool4 = zxz_terms

# Output the pools
print("Pool 1 (Z terms and ZZ terms):")
print(pool1)

print("\nPool 2 (X terms):")
print(pool2)

print("\nPool 3 (ZX terms and XZ terms):")
print(pool3)

print("\nPool 4 (ZXZ terms):")
print(pool4)

H_tot = construct_total_hamiltonian(N, J, hz, hx)

# Compute commutators
commut_pool1 = commutator(H_tot, pool1)
commut_pool2 = commutator(H_tot, pool2)
commut_pool3 = commutator(H_tot, pool3)
commut_pool4 = commutator(H_tot, pool4)

# Print results
print("Commutator of H_total with Pool 1 (Z terms and ZZ terms):")
print(commut_pool1.to_matrix())
print("\nCommutator of H_total with Pool 2 (X terms):")
print(commut_pool2.to_matrix())
print("\nCommutator of H_total with Pool 3 (ZX terms and XZ terms):")
print(commut_pool3.to_matrix())
print("\nCommutator of H_total with Pool 4 (ZXZ terms):")
print(commut_pool4.to_matrix())

Pool 1 (Z terms and ZZ terms):
SparsePauliOp(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ', 'ZZIII', 'IZZII', 'IIZZI', 'IIIZZ', 'ZIIIZ'],
              coeffs=[ 0.24+0.j,  0.24+0.j,  0.24+0.j,  0.24+0.j,  0.24+0.j, -0.12+0.j,
 -0.12+0.j, -0.12+0.j, -0.12+0.j, -0.12+0.j])

Pool 2 (X terms):
SparsePauliOp(['XIIII', 'IXIII', 'IIXII', 'IIIXI', 'IIIIX'],
              coeffs=[-0.1953125+0.j, -0.1953125+0.j, -0.1953125+0.j, -0.1953125+0.j,
 -0.1953125+0.j])

Pool 3 (ZX terms and XZ terms):
SparsePauliOp(['XIIIZ', 'ZXIII', 'IZXII', 'IIZXI', 'IIIZX', 'XZIII', 'IXZII', 'IIXZI', 'IIIXZ', 'ZIIIX'],
              coeffs=[0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j,
 0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j, 0.5859375+0.j])

Pool 4 (ZXZ terms):
SparsePauliOp(['XZIIZ', 'ZXZII', 'IZXZI', 'IIZXZ', 'ZIIZX'],
              coeffs=[-1.7578125+0.j, -1.7578125+0.j, -1.7578125+0.j, -1.7578125+0.j,
 -1.7578125+0.j])
Commutator of H_total with Pool 1 (Z terms and

In [20]:
### EACH TERMS OF THE HAMILTONIANS's EXPONENTIATED ####

# Electric Hamiltonian terms
##ZZ interaction terms
def func_ZZ(circ, i, j, J, theta):
    """Apply exp(-i * J * theta * Z_i * Z_j) using CNOT and Rz gates."""
    theta = 2 * J * theta
    circ.cx(i, j)
    circ.rz(J*theta/2, j)
    circ.cx(i, j)

# Z term
def func_Z(circ, i, hz, theta):
    """Apply exp(-i * hz * theta * Z_i ) using Rz gate."""
    theta = 2 * hz * theta
    circ.rz(hz*theta/2, i)

# Magnetic Hamiltonian terms
# X term
def func_X(circ, i, hx, t, T, delta_t):
    """Apply exp(-i * delta_t * (t/T) * (hx/16) * X_i) using Rx gate."""
    theta = 2 * delta_t * (t/T) * (hx/16)
    circ.rx(theta, i)

# ZX term
def func_ZX(circ, i, j, hx, theta):
    """Apply exp(i * theta * Z_i X_j) using Rz and CNOT gates."""
    theta = 2 * hx * theta
    circ.h(j)
    circ.cx(i,j)
    circ.rz(theta, j)
    circ.cx(i,j)
    circ.h(j)

#XZ term
def func_XZ(circ, i, j, hx, theta):
    """Apply exp(i * theta * X_i Z_j) using Rz and CNOT gates."""
    theta = 2 * hx * theta
    circ.h(i)
    circ.cx(i,j)
    circ.rz(theta, j)
    circ.cx(i,j)
    circ.h(i)
# zxz term
def func_ZXZ(circ, i, j, k, hx, theta):
    """Apply exp(-i * theta * Z_i X_j Z_k) using CNOT, Rz and other gates."""
    theta = 2 * hx * theta
    circ.cx(i,k)
    circ.h(j)
    circ.cx(j,k)
    circ.rz(theta, k)
    circ.cx(j,k)
    circ.cx(i,k)
    circ.h(j)

### PERIODIC BOUNDARY CONDITION:: FIRST ORDER TROTTER STEP
def nth_trotter_step_first_order(circ, J, hz, hx, T, NT, n):
    # n is the number of trotter steps
    
    # number of qubits
    N = circ.num_qubits

    # step size
    delta_t = T / NT

    # instantaneous time #or we can take any random value between 0 and 1 (instead of 0.5
    t = (n+0.001)*delta_t

    ### PERIODIC BOUNDARY CONDITION ####
    """Apply first term of electric hamiltonian"""
    # Apply ZZ terms (second column: 0-1, 2-3, 4-5, 6-7, etc.)
    for i in range(0, N-1, 2):
        if i+1 < N:
            func_ZZ(circ, i, i+1, J, delta_t)     
    circ.barrier()
     # Apply ZZ terms (first column: 1-2, 3-4, 5-6, 7-8, etc.)
    for i in range(1, N-1, 2):
        if i+1 < N:
            func_ZZ(circ, i, i+1, J, delta_t)
    circ.barrier()
     # Apply ZZ terms (PERIODIC) (last and first terms)
    if N > 1:
        func_ZZ(circ, N - 1, 0, J, delta_t)
    circ.barrier()
    
    """Apply second term of electric hamiltonian"""
    # Apply Z terms
    for i in range(N):
        func_Z(circ, i, hz, delta_t)
    circ.barrier()
    
    """Apply first term of magnetic hamiltonian"""
    # Apply X terms
    for i in range(N):
        func_X(circ, i, hx, t, T, delta_t)
    circ.barrier()
    
    """Apply second term of magnetic hamiltonian"""
    # Apply ZX terms (second column: 0-1, 2-3, 4-5, 6-7, etc.)
    for i in range(0, N-1, 2):
        if i + 1 < N:
            func_ZX(circ, i, i+1, hx, t, T, delta_t)
    # Apply ZX terms (first column: 1-2, 3-4, 5-6, 7-8, etc.)
    circ.barrier()
    for i in range(1, N-1, 2):
        if i+1 < N:
            func_ZX(circ, i, i+1, hx, t, T, delta_t)
    circ.barrier()
    # PERIODIC (last and first terms)
    if N > 1:
        func_ZX(circ, N - 1, 0, hx, t, T, delta_t)
    circ.barrier()

    """Apply third term of magnetic hamiltonian"""
    # Apply XZ terms (second column: 0-1, 2-3, 4-5, 6-7, etc.)
    for i in range(0, N-1, 2):
        if i+1 < N:
            func_XZ(circ, i, i+1, hx, t, T, delta_t)
    # Apply XZ terms (first column: 1-2, 3-4, 5-6, 7-8, etc.)
    circ.barrier()
    for i in range(1, N-1, 2):
        if i+1 < N:
            func_XZ(circ, i, i+1, hx, t, T, delta_t)
    circ.barrier()
    # PERIODIC (last and first terms)
    if N > 1:
        func_XZ(circ, N - 1, 0, hx, t, T, delta_t)
    circ.barrier()
    
    """Apply fourth term of magnetic hamiltonian"""
    # Apply ZXZ terms in overlapping triplets
    for i in range(0, N-2, 3):
        if i+2 < N:
            func_ZXZ(circ, i, i+1, i+2, hx, t, T, delta_t)
    circ.barrier()
    for i in range(1, N-2, 3):
        if i+2 < N:
            func_ZXZ(circ, i, i+1, i+2, hx, t, T, delta_t)
    circ.barrier()
    for i in range(2, N-2, 3):
        if i+2 < N:
            func_ZXZ(circ, i, i+1, i+2, hx, t, T, delta_t)
    circ.barrier()
    # Apply ZXZ term for periodic boundary condition
    if N > 2:
        func_ZXZ(circ, N - 2, N - 1, 0, hx, t, T, delta_t)
        func_ZXZ(circ, N - 1, 0, 1, hx, t, T, delta_t)
    circ.barrier()
    circ.barrier()

    return circ

#### Exact Ground State (using scipy.linalg.eigh and tenpy (DMRG)) & Overlap Calculation

In [45]:
# ### FIND EXACT GROUND STATE OF THE HAMILTONIAN ###
def exact_ground_state(H):
    """
    Find the exact ground state of the Hamiltonian H.

    Parameters:
    - H: The Hamiltonian operator.

    Returns:
    - eigenvals: all the eigenvalues of H
    - eigenvecs: all the eigenvectors of H
    - energy_realgs: The ground state energy.
    - psi_realgs: The ground state vector.
    """

    # Compute Hamiltonian matrix
    H_matrix = H.to_matrix()  # H is a qiskit operator (defined above)

    # Classical ground state calculation (real GS) using scipy.linalg.eigh
    #eigenvals, eigenvecs = eig(H_matrix,right=True)   
    eigenvals, eigenvecs = eigh(H_matrix)
    
    # Find the ground state (eigenvector with the smallest eigenvalue)
    index_gs = np.argmin(eigenvals)
    energy_realgs = eigenvals[index_gs]
    psi_realgs = eigenvecs[:,index_gs] # Extract the eigenvector (column) corresponding to the smallest eigenvalue
    '''Return the eigenvalues and eigenvectors of a complex Hermitian (conjugate symmetric) 
    or a real symmetric matrix. Returns two objects, a 1-D array containing the eigenvalues of a, 
    and a 2-D square array or matrix (depending on the input type) of the 
    corresponding eigenvectors (in columns).'''
    
    # Normalize the ground state vector (optional)
    psi_realgs /= np.linalg.norm(psi_realgs)

    return eigenvals, eigenvecs, energy_realgs, psi_realgs

# ### OVERLAP OF THE TWO STATES ###
# # def calc_overlap(psi_1, psi_2):
# #     psi_1 = Statevector(psi_1)
# #     psi_2 = Statevector(psi_2)
# #     # Convert Statevector to numpy array if it's not already
# #     if isinstance(psi_1, Statevector):
# #         psi_1 = psi_1.data
# #     if isinstance(psi_2, Statevector):
# #         psi_2 = psi_2.data
        
# #     # Convert psi_1(2) to numpy array if it's not already
# #     psi_1 = np.asarray(psi_1)
# #     psi_2 = np.asarray(psi_2)
    
# #     psi_1_normal = psi_1 / np.linalg.norm(psi_1)

# #     psi_2_normal = psi_2 / np.linalg.norm(psi_2)
     
# #     overlap = np.abs(np.dot(psi_1_normal.conj(), psi_2_normal))**2
    
# #     return overlap


# find the overlap
def calc_overlap(psi_final, psi_realgs):
    #psi_final = Statevector(psi_final)
    # Convert psi_final to numpy array if it's not already
    psi_final = np.asarray(psi_final)
    psi_realgs = np.asarray(psi_realgs)
    
    psi_realgs_normal = psi_realgs / np.linalg.norm(psi_realgs)
    
    psi_final_normal = psi_final / np.linalg.norm(psi_final)
    
    overlap = np.abs(np.dot(psi_realgs_normal.conj(), psi_final_normal))**2
    
    return overlap

In [30]:
### TESTING THE ABOVE FUNCTIONS ####
# Parameters
N = 5 #spins

a = 1.0 #lattice
g = 0.8 #coupling

J = -3 * g**2 / 16
hz = 3 * g**2 / 8
hx = -2 / (a * g)**2  

# Hamiltonians
H_E = hamiltonian_elec(N,J,hx ) #electric
H_M = hamiltonian_mag(N, hz) #magnetic
H_tot = H_E + H_M #total

# Total Hamiltonian Solutions: Energies and States
eigenvals, eigenvecs, energy_realgs, psi_realgs = exact_ground_state(H_tot)

# Electric Hamiltonian Solutions: Classical / Exact
eigenvals_ele, eigenvecs_ele, energy_realgs_ele, psi_realgs_ele = exact_ground_state(H_E)

print("Total Energies: ", eigenvals[0], eigenvals[1], eigenvals[2], eigenvals[3], eigenvals[-1])
print(f"Delta_E (E_1 - E_0) = {(eigenvals[1]-eigenvals[0]):.3f}")

print("Electric Energies: ", eigenvals_ele[0], eigenvals_ele[1], eigenvals_ele[2], eigenvals_ele[3], eigenvals_ele[-1])


print("Exact GS of H_tot: ", psi_realgs)

print("Exact GS of H_E: ", psi_realgs_ele)

## ##checking the correspondance of eigenvalues and eigenvectors
results_tot = [np.allclose(np.dot(H_tot.to_matrix(), eigenvecs[:,i]), eigenvals[i] * eigenvecs[:,i]) for i in range(eigenvals.size)]
results_ele = [np.allclose(np.dot(H_E.to_matrix(), eigenvecs_ele[:,i]), eigenvals_ele[i] * eigenvecs_ele[:,i]) for i in range(eigenvals.size)]
print("Eigenvalue and Eigenvector Test: ", results_tot, results_ele)

Total Energies:  -16.22767571787667 -9.503679986931974 -9.501234555630589 -9.501234555630528 15.074697752035702
Delta_E (E_1 - E_0) = 6.724
Electric Energies:  -16.225 -9.495000000000001 -9.495 -9.495 15.025
Exact GS of H_tot:  [ 9.99801101e-01+0.j -8.91728560e-03+0.j -8.91728560e-03+0.j
 -1.64909704e-04+0.j -8.91728560e-03+0.j  7.95651567e-05+0.j
 -1.64909704e-04+0.j -3.05144871e-06+0.j -8.91728560e-03+0.j
  7.95651567e-05+0.j  7.95651567e-05+0.j  1.47201394e-06+0.j
 -1.64909704e-04+0.j  1.47201394e-06+0.j -3.05144871e-06+0.j
 -5.64868045e-08+0.j -8.91728560e-03+0.j -1.64909704e-04+0.j
  7.95651567e-05+0.j -3.05144871e-06+0.j  7.95651567e-05+0.j
  1.47201394e-06+0.j  1.47201394e-06+0.j -5.64868045e-08+0.j
 -1.64909704e-04+0.j -3.05144871e-06+0.j  1.47201394e-06+0.j
 -5.64868045e-08+0.j -3.05144871e-06+0.j -5.64868045e-08+0.j
 -5.64868045e-08+0.j  2.16890758e-09+0.j]
Exact GS of H_E:  [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.

### Hamiltonian Density

The hamiltonian density is,
$ H_i = h_z Z_i + J/2 (Z_i Z_{i+1} + Z_{i-1} Z_i) + hx/16 * (X_i - 3 Z_{i-1} X_i - 3 X_i Z_{i+1} + 9 Z_{i-1} X_i Z_{i+1}) $

so, 
$ \braket{H_i} = h_z \braket{Z_i} + J/2 ( \braket{Z_i Z_{i+1}} + \braket{Z_{i-1} Z_i}) + hx/16 * \braket{X_i} - 3 * hx/16 * \braket{Z_{i-1} X_i} - 3 *  hx/16 * \braket{X_i Z_{i+1}} + 9 *  hx/16 * \braket{Z_{i-1} X_i Z_{i+1}} \, . $

In [31]:
# Define the local Hamiltonian density operator: Periodic Boundary Condition
def hamiltonian_density(i, J, hz, hx, N):
    """
    Constructs the Hamiltonian density operator for a single site i.

    Parameters: 
    i (int): Site index
    J (float): constant
    hz (float): another constant
    hx (float): another constant
    N (int): Number of spins

    Returns:
    SparsePauliOp: SparsePauliOp object representing the Hamiltonian density for site i
    """
    pauli_list = []
    coeffs = []
    
    # Term: hz * Z_i
    z_term = ['I'] * N
    z_term[i] = 'Z'
    pauli_list.append(Pauli(''.join(z_term)))
    coeffs.append(hz)

    # Term: J/2 * (Z_i Z_{i+1} + Z_{i-1} Z_i)
    zz_term_1 = ['I'] * N
    zz_term_1[i] = 'Z'
    zz_term_1[(i + 1) % N] = 'Z'
    pauli_list.append(Pauli(''.join(zz_term_1)))
    coeffs.append(J / 2)  
    zz_term_2 = ['I'] * N
    zz_term_2[(i - 1) % N] = 'Z'
    zz_term_2[i] = 'Z'
    pauli_list.append(Pauli(''.join(zz_term_2)))
    coeffs.append(J / 2)

    # Term: hx/16 * X_i 
    x_term = ['I'] * N
    x_term[i] = 'X'
    pauli_list.append(Pauli(''.join(x_term)))
    coeffs.append(hx / 16)

    # Term: (- 3 * hx/16) * Z_{i-1}  X_i 
    zx_term = ['I'] * N
    zx_term[(i - 1) % N] = 'Z'
    zx_term[i] = 'X'
    pauli_list.append(Pauli(''.join(zx_term)))
    coeffs.append(-3 * hx / 16)

    # Term: (- 3 * hx / 16) X_i Z_{i+1} 
    xz_term = ['I'] * N
    xz_term[i] = 'X'
    xz_term[(i + 1) % N] = 'Z'
    pauli_list.append(Pauli(''.join(xz_term)))
    coeffs.append(-3 * hx / 16)

    # Term: (9*hx / 16) Z_{i-1} X_i Z_{i+1})
    zxz_term = ['I'] * N
    zxz_term[(i - 1) % N] = 'Z'
    zxz_term[i] = 'X'
    zxz_term[(i + 1) % N] = 'Z'
    pauli_list.append(Pauli(''.join(zxz_term)))
    coeffs.append(9 * hx / 16)

    # Create SparsePauliOp for the Hamiltonian density
    H_density = SparsePauliOp(pauli_list, coeffs)

    return H_density